# Lấy thông tin khoá học


- Tư duy như sau: 
 + Cần lấy log của khoá học
 + Cần lấy cấu trúc, thông tin khoá học
-> API get khoá học
-> API get danh sách thành viên trong khoá học (giáo viên và học sinh)
-> API log của từng người trong khoá học

In [3]:
import requests
import json
import pandas as pd
from IPython.display import display, HTML
from bs4 import BeautifulSoup

In [ ]:


# Cấu hình Moodle API
moodle_url = "http://localhost:8100/webservice/rest/server.php"
api_token = ""

def call_moodle_api(wsfunction, params=None):
    """
    Gọi Moodle Web Service API với hàm và tham số được chỉ định.

    Args:
        wsfunction (str): Tên hàm Web Service.
        params (dict, optional): Tham số bổ sung cho API.

    Returns:
        dict or list: Dữ liệu JSON trả về từ API hoặc None nếu có lỗi.
    """
    default_params = {
        'wstoken': api_token,
        'wsfunction': wsfunction,
        'moodlewsrestformat': 'json'
    }
    if params:
        default_params.update(params)
    
    try:
        response = requests.get(moodle_url, params=default_params)
        if response.status_code != 200:
            print(f"❌ Lỗi HTTP: {response.status_code}")
            print("Nội dung phản hồi:", response.text)
            return None
        
        data = response.json()
        if isinstance(data, dict) and 'exception' in data:
            print(f"❌ Lỗi API: {data['errorcode']}")
            print(f"Thông điệp: {data['message']}")
            return None
        
        return data
    
    except requests.exceptions.RequestException as e:
        print(f"❌ Lỗi khi gửi yêu cầu: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"❌ Lỗi phân tích JSON: {e}")
        print("Nội dung phản hồi:", response.text)
        return None

def get_all_courses():
    """
    Lấy danh sách tất cả các khóa học.

    Returns:
        list: Danh sách các khóa học hoặc None nếu có lỗi.
    """
    data = call_moodle_api('core_course_get_courses')
    return data if data else []

def get_course_contents(courseid):
    """
    Lấy chi tiết nội dung của một khóa học, bao gồm section và module.

    Args:
        courseid (int): ID của khóa học.

    Returns:
        list: Danh sách các section hoặc None nếu có lỗi.
    """
    return call_moodle_api('core_course_get_contents', {'courseid': courseid})

def get_lesson_details(lessonid):
    """
    Lấy chi tiết một bài học (lesson).

    Args:
        lessonid (int): ID của bài học.

    Returns:
        dict: Thông tin bài học hoặc None nếu có lỗi.
    """
    return call_moodle_api('mod_lesson_get_lesson', {'lessonid': lessonid})

def get_lesson_pages(lessonid):
    """
    Lấy danh sách các trang trong một bài học.

    Args:
        lessonid (int): ID của bài học.

    Returns:
        list: Danh sách các trang hoặc None nếu có lỗi.
    """
    return call_moodle_api('mod_lesson_get_pages', {'lessonid': lessonid})

def get_list_of_users_in_course(courseid):
    """
    Lấy danh sách người dùng trong một khóa học.

    Args:
        courseid (int): ID của khóa học.

    Returns:
        list: Danh sách người dùng hoặc None nếu có lỗi.
    """
    return call_moodle_api('core_enrol_get_enrolled_users', {'courseid': courseid})


### Danh sách người tham gia khoá học (cả giáo viên và học sinh)

In [5]:
courseid = 5
users = get_list_of_users_in_course(courseid)
if users:
    for user in users:
         print(f"ID: {user['id']}, Tên: {user['fullname']}, Email: {user['email']}") 

❌ Lỗi API: invalidtoken
Thông điệp: Invalid token - token not found


### Cấu trúc khoá học


In [7]:
courseid = 5
data = get_course_contents(courseid)
print(json.dumps(data, indent=2, ensure_ascii=False))

[
  {
    "id": 33,
    "name": "General",
    "visible": 1,
    "summary": "",
    "summaryformat": 1,
    "section": 0,
    "hiddenbynumsections": 0,
    "uservisible": true,
    "component": null,
    "itemid": null,
    "modules": [
      {
        "id": 52,
        "url": "http://localhost:8100/mod/forum/view.php?id=52",
        "name": "Announcements",
        "instance": 4,
        "contextid": 80,
        "visible": 1,
        "uservisible": true,
        "visibleoncoursepage": 1,
        "modicon": "http://localhost:8100/theme/image.php?theme=boost&component=forum&rev=1753845787&image=monologo&filtericon=1",
        "modname": "forum",
        "purpose": "collaboration",
        "branded": false,
        "modplural": "Forums",
        "availability": null,
        "indent": 0,
        "onclick": "",
        "afterlink": null,
        "activitybadge": [],
        "customdata": "{\"trackingtype\":\"1\"}",
        "noviewlink": false,
        "completion": 0,
        "downloadcon